# Vehicle Speed Check
### Complete # TODO

# Requirements

In [7]:
import cv2
!pip install dlib
import dlib
import time
import math
import os

  Using cached dlib-20.0.0.tar.gz (3.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dlib: filename=dlib-20.0.0-cp311-cp311-macosx_15_0_arm64.whl size=2932509 sha256=56e629469ebdde99c65475462bcb1638467e943283698c80c6ac715076b2cd20
  Stored in directory: /Users/tahamajs/Library/Caches/pip/wheels/fa/e4/82/70875120d8e3f04c437bff144da98429352b3b026553d77b50
Successfully built dlib


# Loading Model

In [ ]:

MODEL_ADDRESS = os.path.join("models", "myhaar.xml") 

VIDEO_ADDRESS = os.path.join("videos", "cars.mp4")

print(f"Loading Haar Cascade model from: {MODEL_ADDRESS}")
if not os.path.exists(MODEL_ADDRESS):
    print(f"Error: Haar Cascade model file not found at {MODEL_ADDRESS}")
    carCascade = None # TODO
else:
    carCascade = cv2.CascadeClassifier(MODEL_ADDRESS) # TODO
    if carCascade.empty():
        print(f"Error: Failed to load Haar Cascade model from {MODEL_ADDRESS}. File might be corrupted or invalid.")
        carCascade = None 

print(f"Loading video from: {VIDEO_ADDRESS}")
if not os.path.exists(VIDEO_ADDRESS):
    print(f"Error: Video file not found at {VIDEO_ADDRESS}")
    video = None # TODO
else:
    video = cv2.VideoCapture(VIDEO_ADDRESS) # TODO
    if not video.isOpened():
        print(f"Error: Could not open video file {VIDEO_ADDRESS}.")
        video = None

WIDTH = 1280  
HEIGHT = 720  

Loading Haar Cascade model from: models/myhaar.xml
Loading video from: videos/cars.mp4


This Python script sets up paths and attempts to load a Haar Cascade model and a video file using OpenCV.
It defines `MODEL_ADDRESS` for the Haar model ("models/myhaar.xml") and `VIDEO_ADDRESS` for the video ("videos/cars.mp4"), then tries to load them, checking if files exist and if loading is successful.
If the Haar model file isn't found or fails to load, `carCascade` is set to `None`.
Similarly, if the video file is missing or cannot be opened, `video` is set to `None`.
Finally, it defines `WIDTH` and `HEIGHT` constants (1280x720) for video processing.

# Speed Detection

In [ ]:
def estimateSpeed(location1, location2):
    d_pixels = math.sqrt(
        math.pow(location2[0] - location1[0], 2)
        + math.pow(location2[1] - location1[1], 2)
    )
    ppm = 8.8 
    d_meters = d_pixels / ppm
    fps = 18  
    speed = d_meters * fps * 3.6  
    return speed

def trackMultipleObjects():
    rectangleColor = (0, 255, 0) 
    frameCounter = 0 
    currentCarID = 0 
    fps = 0 

    carTracker = {} 
    carLocation1 = {} 
    carLocation2 = {} 
    speed = [None] * 1000  

    out = cv2.VideoWriter(
        "outpy.avi", cv2.VideoWriter_fourcc("M", "J", "P", "G"), 10, (WIDTH, HEIGHT)
    )

    while True:
        start_time = time.time() 
        rc, image = video.read()  
        if type(image) == type(None):
            break

        image = cv2.resize(image, (WIDTH, HEIGHT)) 
        resultImage = image.copy() 

        frameCounter = frameCounter + 1  

        carIDtoDelete = [] 

        for carID in carTracker.keys():
            trackingQuality = carTracker[carID].update(image)
            if trackingQuality < 7:
                carIDtoDelete.append(carID)

        for carID in carIDtoDelete:
            print("Removing carID " + str(carID) + " from list of trackers.")
            print("Removing carID " + str(carID) + " previous location.")
            print("Removing carID " + str(carID) + " current location.")
            carTracker.pop(carID, None)
            carLocation1.pop(carID, None)
            carLocation2.pop(carID, None)

        if not (frameCounter % 10):
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            cars = carCascade.detectMultiScale(gray, 1.1, 13, 18, (24, 24))
            for _x, _y, _w, _h in cars:
                x = int(_x)
                y = int(_y)
                w = int(_w)
                h = int(_h)
                x_bar = x + 0.5 * w
                y_bar = y + 0.5 * h
                matchCarID = None

                for carID in carTracker.keys():
                    trackedPosition = carTracker[carID].get_position()
                    t_x = int(trackedPosition.left())
                    t_y = int(trackedPosition.top())
                    t_w = int(trackedPosition.width())
                    t_h = int(trackedPosition.height())
                    t_x_bar = t_x + 0.5 * t_w
                    t_y_bar = t_y + 0.5 * t_h
                    if (
                        (t_x <= x_bar <= (t_x + t_w))
                        and (t_y <= y_bar <= (t_y + t_h))
                        and (x <= t_x_bar <= (x + w))
                        and (y <= t_y_bar <= (y + h))
                    ):
                        matchCarID = carID

                if matchCarID is None:
                    print("Creating new tracker " + str(currentCarID))
                    tracker = dlib.correlation_tracker()
                    tracker.start_track(image, dlib.rectangle(x, y, x + w, y + h))
                    carTracker[currentCarID] = tracker
                    carLocation1[currentCarID] = [x, y, w, h]
                    currentCarID = currentCarID + 1

        for carID in carTracker.keys():
            trackedPosition = carTracker[carID].get_position()
            t_x = int(trackedPosition.left())
            t_y = int(trackedPosition.top())
            t_w = int(trackedPosition.width())
            t_h = int(trackedPosition.height())
            cv2.rectangle(
                resultImage, (t_x, t_y), (t_x + t_w, t_y + t_h), rectangleColor, 4
            )
            carLocation2[carID] = [t_x, t_y, t_w, t_h]

        end_time = time.time()  
        if not (end_time == start_time):
            fps = 1.0 / (end_time - start_time)

        for i in carLocation1.keys():
            if frameCounter % 1 == 0:
                [x1, y1, w1, h1] = carLocation1[i]
                [x2, y2, w2, h2] = carLocation2[i]
                carLocation1[i] = [x2, y2, w2, h2]
                if [x1, y1, w1, h1] != [x2, y2, w2, h2]:
                    if (speed[i] == None or speed[i] == 0) and y1 >= 275 and y1 <= 285:
                        speed[i] = estimateSpeed([x1, y1, w1, h1], [x2, y2, w2, h2])
                    if speed[i] != None and y1 >= 180:
                        cv2.putText(
                            resultImage,
                            str(int(speed[i])) + " km/hr",
                            (int(x1 + w1 / 2), int(y1 - 5)),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.75,
                            (255, 255, 255),
                            2,
                        )

        cv2.imshow("result", resultImage)
        if cv2.waitKey(33) == 27:
            break

    cv2.destroyAllWindows()

if __name__ == "__main__":
    trackMultipleObjects()


2025-06-02 02:31:51.238 python[97842:4035961] +[IMKClient subclass]: chose IMKClient_Legacy
2025-06-02 02:31:51.239 python[97842:4035961] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Creating new tracker 0
Creating new tracker 1
Creating new tracker 2
Creating new tracker 3
Creating new tracker 4
Creating new tracker 5
Creating new tracker 6
Creating new tracker 7
Creating new tracker 8
Creating new tracker 9
Creating new tracker 10
Creating new tracker 11
Removing carID 0 from list of trackers.
Removing carID 0 previous location.
Removing carID 0 current location.
Removing carID 5 from list of trackers.
Removing carID 5 previous location.
Removing carID 5 current location.
Creating new tracker 12
Removing carID 3 from list of trackers.
Removing carID 3 previous location.
Removing carID 3 current location.
Creating new tracker 13
Removing carID 4 from list of trackers.
Removing carID 4 previous location.
Removing carID 4 current location.
Creating new tracker 14
Creating new tracker 15
Removing carID 15 from list of trackers.
Removing carID 15 previous location.
Removing carID 15 current location.


KeyboardInterrupt: 

: 

This Python script is designed to detect, track, and estimate the speed of cars in a video.

The `estimateSpeed` function takes two location points of a car as input. It calculates the pixel distance between these points, converts it to meters using a predefined pixels-per-meter ratio (ppm = 8.8), and then, using a fixed FPS (18), it computes the speed in kilometers per hour. This estimated speed is then returned.

The `trackMultipleObjects` function processes a video stream frame by frame. It uses a Haar Cascade model (`carCascade`) for initial car detection every 10 frames and dlib correlation trackers for continuous tracking. New trackers are created for newly detected cars, and existing trackers are updated or removed based on tracking quality. For each tracked car, its speed is estimated using `estimateSpeed` when it crosses a virtual line, and this information, along with bounding boxes, is displayed on the output video, which is also saved.